In [4]:
#Load dataset
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f751a0eb-49c1-4054-a8a6-888c9182271b)')' thrown while requesting HEAD https://huggingface.co/datasets/knkarthick/dialogsum/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1198d940-8e1d-4bc8-974e-c10e2765225b)')' thrown while requesting HEAD https://huggingface.co/datasets/knkarthick/dialogsum/resolve/main/README.md
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bc7c8456-3a15-4508-93c5-8bb5fc7a47ba)')' thrown while requesting HEAD https://huggingface.co/datasets/knkarthick/dialogsum/resolve/a968e7aee0602e257935f1321a02e4287f7d5848/dataset_infos.json
Retrying in 1s [Retry 1/5].


In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [3]:
ds['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [ ]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [ ]:
  #tokenization
  def preprocess_function(batch):
    source = batch['dialogue']
    target = batch['summary']
    source_ids = tokenizer(source,truncation=True,padding = 'max_length',max_length = 130)
    target_ids = tokenizer(target,truncation=True,padding = 'max_length',max_length = 130)

    labels = target_ids['input_ids']
    labels = [[(label if label != tokenizer.pad_token_id else - 100)for label in labels_example] for labels_example in labels]

    return {
        "input_ids":source_ids["input_ids"],
        "attention_mask":source_ids["attention_mask"],
        "labels":labels
    }

In [ ]:
df_source = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/bart_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    remove_unused_columns=True,
    save_total_limit=1,        # ✅ keep only the last checkpoint
    save_strategy="epoch",     # ✅ save only once per epoch
    logging_strategy="steps",
    logging_steps=200,         # less frequent logging
    predict_with_generate=True,
    fp16=False
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=df_source["train"],
    eval_dataset=df_source["test"],
    tokenizer=tokenizer,                  # ✅ add tokenizer (needed for seq2seq)
)


/tmp/ipython-input-1132342802.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shreenalathiya42 (shreenalathiya42-auro-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
200,2.025300
400,1.939600
600,1.830800
800,1.916200
1000,1.828700
1200,1.832400
1400,1.769000
1600,1.774200
1800,1.723200
2000,1.722100


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=12460, training_loss=1.4040167898848581, metrics={'train_runtime': 3613.3863, 'train_samples_per_second': 6.897, 'train_steps_per_second': 3.448, 'total_flos': 4897085929881600.0, 'train_loss': 1.4040167898848581, 'epoch': 2.0})

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
eval_results

{'eval_loss': 1.691376805305481,
 'eval_runtime': 57.1411,
 'eval_samples_per_second': 26.251,
 'eval_steps_per_second': 13.125,
 'epoch': 2.0}

In [ ]:
from transformers import GenerationConfig

gen_config = GenerationConfig(
    max_length=100,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)


In [ ]:
model.generation_config.early_stopping = True
model.save_pretrained('/bart_model/model_directory')
tokenizer.save_pretrained('/bart_model/model_directory')

('/bart_model/model_directory/tokenizer_config.json',
 '/bart_model/model_directory/special_tokens_map.json',
 '/bart_model/model_directory/vocab.json',
 '/bart_model/model_directory/merges.txt',
 '/bart_model/model_directory/added_tokens.json',
 '/bart_model/model_directory/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/bart_model/model_directory')
model = AutoModelForSeq2SeqLM.from_pretrained('/bart_model/model_directory')

def summarize(blog_post):
    # Tokenize with a larger max length
    inputs = tokenizer(
        blog_post,
        truncation=True,
        max_length=1024,  # allow longer sequences
        return_tensors='pt'
    )

    # Generate summary
    summary_ids = model.generate(
        inputs['input_ids'],
        generation_config=gen_config
    )


    # Decode tokens
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
blog_post = """
The main problem of one-hot encoding is that each protein has a different length, while all input vectors should be of the same size to be fed into the model. To overcome this issue, sequence padding and truncation are usually applied7,9,12,14. This means establishing a common length for all proteins and then, truncating longer proteins to that length or filling shorter proteins with an “artificial” character up until that length (see Fig. 1A). This process of completing a sequence is called padding and the character used for filling could be any that is not used in the sequences themselves. To this matter, zero character (“0”) is the most commonly used. Padding zeros can be added at any position of the sequence, for example at the N- and C- terminals of the sequences28. In practice, they are usually added at the end7,14. However, details on the concrete steps of sequences padding are often omitted as they are deemed of low importance for the results of the study9,12,17,27. Even when this information is given, there is no proper justification on the padding choice7,14,28. This is partly due to the lack of exhaustive studies on the effect of padding the sequences. Up to our knowledge, the work of Reddy et al.29 is the only study on the effect of sequence padding on deep learning models. It was applied on a NLP sentiment analysis task and only pre- and post- padding types were tested. Since it is a different application domain and the options they test are limited, a more comprehensive study for the case of biological sequences is needed. Likewise, alternative types of padding to those usually implemented (zeros at the end of the sequence, at the beginning or both) have not been yet explored.

"""
summary = summarize(blog_post)
print(f"Summary: {summary}")

`generation_config` default values have been modified to match model-specific defaults: {'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2, 'pad_token_id': 1, 'bos_token_id': 0, 'eos_token_id': [2], 'decoder_start_token_id': 2}. If this is not desired, please set these values explicitly.


Summary: The main problem of one-hot encoding is that each protein has a different length and all input vectors should be of the same size. To overcome this issue, sequence padding and truncation are usually applied7,9,12,14.


In [ ]:
from google.colab import files
import shutil

# Compress the folder
shutil.make_archive("/bart_model/model_directory", 'zip', "/bart_model/model_directory")

# Download the zip
files.download("/bart_model/model_directory.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>